<a href="https://colab.research.google.com/github/igotthisbros/FinanceNewsLLM/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Setup Cell ---
!pip install -q langchain langchain-community chromadb sentence-transformers transformers accelerate newspaper3k pypdf


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
prompt = "Explain the difference between supervised and unsupervised learning in simple terms."
response = pipe(prompt, max_new_tokens=150)
print(response[0]['generated_text'])


Explain the difference between supervised and unsupervised learning in simple terms.



**Solution to Instruction 1:**

Supervised and unsupervised learning are both methods used in machine learning, but they differ in how they approach the learning process.


- **Supervised Learning:** This method involves training a model on a labeled dataset, which means that each training example is paired with an output label. The model learns to predict the output from the input data. After the training process, the model can predict the output for new, unseen data. It's like learning to recognize different types of animals; you're given pictures of animals with labels (e.g., dog, cat, bird) and learn to classify new pictures correctly.


In [4]:
from langchain.chains import RetrievalQA
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-sdk~=1.3

In [5]:


# Model
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

# Pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

# Embeddings + Chroma
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})

# RAG Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Query
question = "Explain what retrieval augmented generation is."
result = qa_chain.run(question)
print(result)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
/tmp/ipython-input-3773682830.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipython-input-3773682830.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-3773682830.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
/tmp/ipython-input-3773682830.py:24: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(question)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: Explain what retrieval augmented generation is.
Helpful Answer:

Retrieval Augmented Generation (RAG) is a technique in the field of natural language processing and machine learning. It combines the strengths of two well-established methods: large language models and information retrieval systems.

Large language models, such as GPT-3 or BERT, generate text based on the input they receive and the patterns they have learned during training. These models can perform various tasks, such as translation, summarization, and content generation. However, they often lack access to specific, real-time information or knowledge not present in their training data.

Information retrieval systems, on the other hand, use algorithms to search and retrieve relevant information from databases or the internet. They can find specific

In [6]:
from newspaper import Article

def scrape_article(url):
  article = Article(url)
  article.download()
  article.parse()
  return article.text


db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

def add_news_to_chroma(url):
  text = scrape_article(url)
  db.add_texts([text], metadatas=[{"source":url}])
  print("Added",url)




In [10]:
%%writefile news_mcp_server.py
import json
from mcp.server.fastmcp import FastMCP
from newspaper import Article

app = FastMCP("news-scraper")

def scrape(url: str) -> str:
    """Scrape news article text using newspaper3k."""
    article = Article(url)
    article.download()
    article.parse()
    return article.text

@app.tool()
def fetch_news(url: str) -> dict:
    """
    MCP tool: fetch a news article and return its text content.
    """
    text = scrape(url)
    return {
        "url": url,
        "content": text,
        "length": len(text)
    }

if __name__ == "__main__":
    app.run()



Overwriting news_mcp_server.py


In [12]:
from mcp.client import MCPClient
import asyncio

async def test():
    client = MCPClient("http://127.0.0.1:3925")
    result = await client.call_tool("fetch_news", {
        "url": "https://www.bbc.com/news/business-123456"
    })
    print(result)

asyncio.run(test())


ImportError: cannot import name 'MCPClient' from 'mcp.client' (/usr/local/lib/python3.12/dist-packages/mcp/client/__init__.py)